In [ ]:
%%capture
import sys

# Añade el directorio principal al path de búsqueda para importar módulos desde esa ubicación
sys.path.insert(0, "..")

import json

import mlflow
import numpy as np
import pandas as pd
import tensorflow as tf
from likelihood.models.deep import AutoClassifier
from likelihood.tools import OneHotEncoder
from mlflow.models.signature import infer_signature
from mlflow.utils.environment import _mlflow_conda_env
from sklearn import datasets
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

is_updated = False
from packaging import version

if version.parse(tf.__version__) > version.parse("2.15.0"):
    is_updated = True

In [ ]:
# Cargar el dataset de cáncer de mama desde sklearn
df = datasets.load_breast_cancer()

# Convertir los datos a un DataFrame de pandas para facilitar la manipulación
df_cancer = pd.DataFrame(data=df.data, columns=df.feature_names)
df_cancer["target"] = df.target  # Añadir la columna de etiquetas 'target'

# OneHotEncoder convierte las etiquetas a formato one-hot encoding
y_encoder = OneHotEncoder()
y = y_encoder.encode(df_cancer["target"].to_list())  # Codificar las etiquetas de la clase (target)
X = df_cancer.drop(
    columns="target"
).to_numpy()  # Extraer las características (sin la columna 'target')
X = np.asarray(X).astype(np.float32)  # Convertir X a tipo float32 para la entrada del modelo
y = np.asarray(y).astype(np.float32)  # Convertir y a tipo float32

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de clasificación automática con las especificaciones dadas
model = AutoClassifier(
    input_shape_parm=X.shape[1],  # El número de características de entrada (columnas de X)
    num_classes=y.shape[1],  # El número de clases (salidas) del modelo
    units=17,  # Número de unidades en las capas ocultas
    activation="selu",  # Función de activación de las capas ocultas
    l2_reg=0.0,
)

if is_updated:
    model = model._main_model

In [ ]:
dummy_input = tf.convert_to_tensor(tf.random.normal([1, X.shape[1]]))
# Guardamos dummy_input para generar el sample-request.json más adelante
sample_request = {
    "input_data": {
        "columns": list(range(X.shape[1])),
        "index": list(range(2)),
        "data": X_train[:2].tolist(),
    }
}
with open("sample-request.json", "w") as f:
    json.dump(sample_request, f)
model(dummy_input)

# Conda environment
custom_env = _mlflow_conda_env(
    additional_conda_deps=None,
    additional_pip_deps=["likelihood[full]", "azureml-inference-server-http"],
    additional_conda_channels=None,
)

# Sample
input_sample = X_train[0].reshape(1, -1)
output_sample = model.predict(input_sample)

# Compilación del modelo
model.compile(
    optimizer="adam",  # Optimizador Adam
    loss=tf.keras.losses.CategoricalCrossentropy(),  # Función de pérdida para clasificación multiclase
    metrics=[
        tf.keras.metrics.AUC(),  # Añadir la métrica AUC para un análisis más completo
        tf.keras.metrics.CategoricalAccuracy(),  # Métrica de precisión categórica
        tf.keras.metrics.F1Score(threshold=0.5),  # Métrica F1 con threshold
    ],
)


# Definir callback para registrar las métricas en cada época
class LogMetricsCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        # Loggear las métricas como valores escalares
        mlflow.log_metric("loss", float(logs["loss"]), step=epoch)
        mlflow.log_metric("val_loss", float(logs["val_loss"]), step=epoch)
        mlflow.log_metric("accuracy", float(logs["categorical_accuracy"]), step=epoch)
        mlflow.log_metric("val_accuracy", float(logs["val_categorical_accuracy"]), step=epoch)
        mlflow.log_metric("auc", float(logs["auc"]), step=epoch)
        mlflow.log_metric("val_auc", float(logs["val_auc"]), step=epoch)
        mlflow.log_metric("f1_score", float(logs["f1_score"][0]), step=epoch)
        mlflow.log_metric("val_f1_score", float(logs["val_f1_score"][0]), step=epoch)


early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

if is_updated:
    checkpoint = ModelCheckpoint(
        "best_model.keras",
        monitor="val_loss",
        save_best_only=True,
        save_weights_only=False,
        mode="min",
        verbose=1,
    )
else:
    checkpoint = ModelCheckpoint(
        "best_model",
        monitor="val_loss",
        save_best_only=True,
        save_weights_only=False,
        mode="min",
        verbose=1,
        save_format="tf",
    )

In [ ]:
with mlflow.start_run():
    # Entrenar el modelo con los datos, usando 15 épocas y 20% de los datos para validación
    history = model.fit(
        X_train,
        y_train,
        epochs=15,
        batch_size=32,
        validation_split=0.2,
        verbose=1,  # Para obtener información durante el entrenamiento
        callbacks=[
            LogMetricsCallback(),
            early_stopping,
            checkpoint,
        ],  # Usar el callback para registrar las métricas
    )
    print("Modelo entrenado")

    # Inferir la firma del modelo
    signature = infer_signature(X_train, model.predict(X_train))

    # Loggear el modelo con la firma personalizada
    mlflow.tensorflow.log_model(
        model,
        artifact_path="model_clf",
        signature=signature,
        input_example=X_train[:1],  # Tomar un ejemplo de entrada
        conda_env=custom_env,  # Asegúrate de tener un entorno Conda adecuado
    )